In [55]:
import os
import sys
import json
from glob import glob
# from google.colab import drive
# from google.colab import files

# ROOT = "/content/drive"
# ROOT = os.getcwd()    
# 만약 함수로 짜야한다면, 최상단에 main.py로 작성할 때 주로 사용하는 경로이나, 주피터에선 ㄴㄴ

ROOT = "/Volumes/WORK/Google/Colab-Server-VM/Waste-Recycling-Image-Segmentation"
# os로 경로 자유자재로 설정하는 부분은 다시 봐야함

# try:
#     drive.mount(ROOT)
# except:
#     drive.mount(ROOT, force_remount=True)

# TASK_PATH = "MyDrive/Task/plastic-segmentation/Data"
TASK_PATH = "assets/data/plastic-segmentation"

TRN_PATH = os.path.join(ROOT, TASK_PATH, "Train")
TST_PATH = os.path.join(ROOT, TASK_PATH, "Test")

In [56]:
PCLS = ["PE", "PET", "PP"]  # , "PS" 테스트 데이터가 이상하게 들어가서 우선은 리스트에 넣지 않고 빼도록 함

trn_paths = {"image_path" : [], "annot_path": []}
tst_paths = {"image_path" : [], "annot_path": []}

for fpath in [TRN_PATH, TST_PATH]:
    for pcls in PCLS:
        img_path = os.path.join(fpath, pcls)
        img_paths = sorted(glob(img_path + "/*.jpg"))
        if fpath.split('/')[-1]=="Train":
            annot_path = os.path.join(fpath, "annotation", pcls)
            trn_paths["image_path"].extend(img_paths)
            trn_paths["annot_path"].extend(sorted(glob(annot_path + "/*.json")))
        elif fpath.split('/')[-1]=="Test":
            annot_path = os.path.join(fpath, "annotations", pcls)
            tst_paths["image_path"].extend(img_paths)
            tst_paths["annot_path"].extend(sorted(glob(annot_path + "/*.json")))

In [57]:
import pandas as pd
pd.DataFrame(trn_paths).iloc[0,:].apply(lambda x:x.split('/')[-1])

image_path     PE_074_10.jpg
annot_path    PE_074_10.json
Name: 0, dtype: object

In [58]:
import pandas as pd
# trn_paths["datas"] = list(map(lambda x: (x.split('/')[-4]).lower(),trn_paths["annot_path"]))
# tst_paths["datas"] = list(map(lambda x: (x.split('/')[-4]).lower(),tst_paths["annot_path"]))
## 이미 변수로 구분해서 필요없음

trn_paths["classes"] = list(map(lambda x: (x.split('/')[-1]).split('_')[0],trn_paths["annot_path"]))
trn_paths["feats_1"] = list(map(lambda x: (x.split('/')[-1]).split('_')[1],trn_paths["annot_path"]))
trn_paths["feats_2"] = list(map(lambda x: (x.split('/')[-1]).split('_')[-1][:-5],trn_paths["annot_path"]))

tst_paths["classes"] = list(map(lambda x: (x.split('/')[-1]).split('_')[0],tst_paths["annot_path"]))
tst_paths["feats_1"] = list(map(lambda x: (x.split('/')[-1]).split('_')[1],tst_paths["annot_path"]))
tst_paths["feats_2"] = list(map(lambda x: (x.split('/')[-1]).split('_')[-1][:-5],tst_paths["annot_path"]))

pd.DataFrame(trn_paths)
# yml
# pyenv conda 어떤 것을 설치해야하는지
# 모델을 이미지 특성별 돌려야 함은 이전에 회의한 부분.
# 세부 모델을 몇 단계까지 고려해야할지는 데이터를 중간중간 확인하면 진행해야 함
# 그렇다면 우선적으로 처리해야 할 부분은...?
# model/__main__.py
#           |__init__.py
#           |__PE
#           |__...
#           |__PS
# feats_1과 feats_2를 어떻게 받고 처리할지는 우선 위의 flow를 다 작성한 뒤, 진행
# feats_1 == metainfo_id

,image_path,annot_path,classes,feats_1,feats_2
0,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PE,074,10
1,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PE,074,100
2,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PE,074,101
3,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PE,074,102
4,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PE,074,103
...,...,...,...,...,...
2995,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PP,058,8929
2996,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PP,058,8995
2997,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PP,058,908
2998,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,/Volumes/WORK/Google/Colab-Server-VM/Waste-Rec...,PP,058,910


In [59]:
# train annotations[0]
with open(trn_paths["annot_path"][0], "rb") as jsf:
    # jsd = json.loads(jsf.read())
    jsd = json.load(jsf)

# jsd.keys()
# dict_keys(['images', 'categories', 'annotations', 'info', 'metainfo'])

In [77]:
'''
# images
width = image total width
height = image total height
id -->> annotations/image_id
file_name = image file name
** annotation 검증 단계에서만 사용될 듯
++ annotation 활용까지는 config로 ㄱ?
'''
print(pd.DataFrame(jsd['images']))

''' 
# categories
object's metainfo
있어야 하나??? 이미 폴더 구조로 정보를 다 주었는데,,,이렇게 넣은 건...각각 따로 봐라??
아니면 오브젝트 전체의 집합......도와줘요 연제님
'''
print(pd.DataFrame(jsd['categories']))

''' 
# info

별로 중요한 변수는 아니라고 생각되나, 'year'를 추출해야할 수 있을 수 있음
예, 해가 바뀌며 재활용 분류 기준이 달라지거나, 호칭 및 명칭이 달라진 경우. (없을 것 같지만,,,)
'''
print(pd.Series(jsd['info'], name='info'))

   width  height  id      file_name
0   2048    2048   1  PE_074_10.jpg
   id name
0   1  pet
1   2   ps
2   3   pp
3   4   pe
year                2021
version              1.0
contributor    datamaker
Name: info, dtype: object


In [60]:
'''
# annotations
iscrowd는 maskrcnn pytorch documents에서의 튜토리얼에 나왔었음
각 항목들을 다른 key값을 동시에 보면서 연관지어야 할 것으로 보임.
id <<-- object_cnt / object_num 정도로 해석하면 될 듯?
image_id == images.id
category_id = class id -->> catId
metainfo_id == int(file_name.split('_')[1])
bbox : [xmean, ymean, width, height]
'''
pd.DataFrame(jsd['annotations'])

,id,image_id,category_id,metainfo_id,segmentation,bbox,ignore,iscrowd,area
0,1,1,4,74,"[[890, 837, 883, 841, 877, 847, 872, 854, 868,...","[792, 837, 191, 221]",0,0,25260.0
1,2,1,4,74,"[[1383, 765, 1377, 767, 1371, 770, 1365, 772, ...","[1285, 765, 169, 193]",0,0,22093.5
2,3,1,4,74,"[[1149, 409, 1142, 415, 1140, 422, 1138, 429, ...","[1109, 409, 148, 186]",0,0,22911.5


In [90]:
import numpy as np

for num in range(3):
    print(f"{num+1} 번째 객체의 segmentation 지점은 총 {len(np.array(jsd['annotations'][num]['segmentation']).reshape(-1))} 입니다.")

1 번째 객체의 segmentation 지점은 총 158 입니다.
2 번째 객체의 segmentation 지점은 총 174 입니다.
3 번째 객체의 segmentation 지점은 총 160 입니다.


In [69]:
'''
# metainfo
: id, name, category_id, container(T/F)
** name 나누면 매칭되는 값이 metainfo에 많음.

name => .split('_')
name = map(lambda x: x.join(' '))
'''
len(jsd['metainfo'])
metainfo_df = pd.DataFrame(jsd['metainfo'])
# metatinfo_df['classes'] = metainfo_df['name']
for col in metainfo_df.columns:
    print(col, '\t\t', list(metainfo_df[col].unique()) if len(metainfo_df[col].unique())<=10 else len(metainfo_df[col].unique()))
    
metainfo_df[metainfo_df['id']==74]

id 		 97
name 		 97
category_id 		 [1, 2, 3, 4]
container 		 [True, False]
transparent 		 [True, False]
shape 		 [0, 3, 1, 2]
size 		 [0, 1, 2]
compress 		 [0, 1, 2]
label 		 [True, False]


,id,name,category_id,container,transparent,shape,size,compress,label
67,74,pe_식품용기류_불투명_원형_소_비압축_라벨유,4,True,False,1,1,0,True


## 추가적으로 봐야할 부분
- 다른 클래스나 상위 폴더 중 test의 annotation도 동일하게 적용이 되는지
- 특성할 수 있는 부분은 경로나 metainfo의 name을 학습할 수 있도록 각각 numeric한 값으로 정리한 내용인데 데이터가 제대로 매칭된 게 맞는지
- 제공된 segmentation, bbox 등의 수치 값의 오차 범위는 어떻게 되는지

## poetry YML 참고할 라이브러리(정확히는 확인해봐야 할 라이브러리)

| pkg | version | defs |
|:----|:--------:|:------|
| pyyml | 0.0.2 | Use python in yaml |
| yml | 0.0.1 | A fast, safe, pure Python YAML and JSON parser |
| ci-yml | 0.3.3 | A .ci.yml parser for Gitlab-CI |
| yaml2env | 0.0.1 | A package to convert yaml file to exportable environmental variable |

**목적 :  ipynb로 제출할 파일 환경 구성**